In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

D:\Matteo\Programmi\Anaconda\envs\tf_gpu_13\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Matteo\Programmi\Anaconda\envs\tf_gpu_13\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Matteo\Programmi\Anaconda\envs\tf_gpu_13\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Matteo\Programmi\Anaconda\envs\tf_gpu_13\lib\s

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14785337599111505167
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 3263086592
locality {
  bus_id: 1
}
incarnation: 13578535081011926090
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0"
]


In [2]:
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Conv2D, MaxPooling2D

# from keras.backend.tensorflow_backend import set_session
# import tensorflow as tf
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = True  # to log device placement (on which device the operation ran)
# sess = tf.Session(config=config)
# set_session(sess)  # set this TensorFlow session as the default session for Keras

data_path = '../sim_data/run1/'

Using TensorFlow backend.


In [3]:
logs = pd.read_csv(data_path + 'driving_log.csv')

logs = logs.values

images = []
measurements = []

images_path = data_path + 'IMG/'

for i in tqdm(range(len(logs))):
    line = logs[i]
    center_path = line[0]
    center_path = images_path + center_path.split('/')[-1]
    center_img = cv2.imread(center_path)
    images.append(center_img)
    
    measure = float(line[3])
    measurements.append(measure)
    
X_train = np.array(images)
y_train = np.array(measurements)
    
    

100%|█████████████████████████████████████████████████████████████████████████████| 4443/4443 [00:04<00:00, 939.50it/s]


In [7]:
model = Sequential()

# Normalization
model.add(Lambda(lambda x: x / 255 - 0.5, input_shape=(160, 320, 3)))

model.add(Conv2D(6,(5,5), activation="relu", input_shape=(160, 320, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(6,(5,5), activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(8))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model.h5')

Train on 3554 samples, validate on 889 samples
Epoch 1/5
3554/3554 [==============================] - 12s 3ms/step - loss: 6.6737 - val_loss: 0.0308
Epoch 2/5
3554/3554 [==============================] - 9s 2ms/step - loss: 0.0242 - val_loss: 0.0293
Epoch 3/5
3554/3554 [==============================] - 9s 2ms/step - loss: 0.0223 - val_loss: 0.0274
Epoch 4/5
3554/3554 [==============================] - 9s 2ms/step - loss: 0.0208 - val_loss: 0.0258
Epoch 5/5
3554/3554 [==============================] - 9s 2ms/step - loss: 0.0195 - val_loss: 0.0243
